In [1]:
reset -fs

In [2]:
import pandas as pd
import re
import string

from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
from nltk.stem.porter import PorterStemmer

from sklearn.naive_bayes import MultinomialNB

from sklearn.metrics import accuracy_score

In [3]:
# Comments File
comments = 'Data/CommentsApril2018.csv'

In [4]:
# Read in File
comm = pd.read_csv(comments); comm.head()

/anaconda3/envs/ml/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3018: DtypeWarning: Columns (32) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,approveDate,articleID,articleWordCount,commentBody,commentID,commentSequence,commentTitle,commentType,createDate,depth,...,status,timespeople,trusted,typeOfMaterial,updateDate,userDisplayName,userID,userLocation,userTitle,userURL
0,1524594282,5adf6684068401528a2aa69b,781.0,How could the league possibly refuse this offe...,26853969.0,26853969.0,<br/>,comment,1524594011,1.0,...,approved,1,0,News,1524594282,Christopher Rillo,46566740.0,San Francisco,NaN,NaN
1,1524594252,5adf6684068401528a2aa69b,781.0,"So then the execs can be like ""yeah...we will ...",26853699.0,26853699.0,<br/>,comment,1524593146,1.0,...,approved,1,0,News,1524594252,Matt Brand,64324866.0,"Williamsburg, Brooklyn",NaN,NaN
2,1524594250,5adf6684068401528a2aa69b,781.0,I would not want to play chess against these c...,26853677.0,26853677.0,<br/>,comment,1524593032,1.0,...,approved,1,0,News,1524594250,Joseph,78105093.0,"Fayetteville, AR",NaN,NaN
3,1524593431,5adf6684068401528a2aa69b,781.0,Could the cheerleaders join the Actors' Equity...,26853784.0,26853784.0,<br/>,comment,1524593426,1.0,...,approved,0,0,News,1524593431,Stephen,81939618.0,"Phoenix, AZ",NaN,NaN
4,1524595048,5adf653f068401528a2aa697,656.0,Seeking conclusions which support preconceived...,26854236.0,26854236.0,<br/>,comment,1524595043,1.0,...,approved,1,0,News,1524595048,Paul Zorsky,58642997.0,Texas,NaN,NaN


In [13]:
comm.columns

Index(['approveDate', 'articleID', 'articleWordCount', 'commentBody',
       'commentID', 'commentSequence', 'commentTitle', 'commentType',
       'createDate', 'depth', 'editorsSelection', 'inReplyTo', 'newDesk',
       'parentID', 'parentUserDisplayName', 'permID', 'picURL', 'printPage',
       'recommendations', 'recommendedFlag', 'replyCount', 'reportAbuseFlag',
       'sectionName', 'sharing', 'status', 'timespeople', 'trusted',
       'typeOfMaterial', 'updateDate', 'userDisplayName', 'userID',
       'userLocation', 'userTitle', 'userURL'],
      dtype='object')

In [14]:
comm.recommendations.unique()

array([   0,    3,    2, ..., 1877, 1137, 1307])

In [5]:
def vec_train_test(data, labels, vectorizer, algorithm, random_state=28):
    # Create train/test split with labels
    train_data, test_data, train_target, test_target = train_test_split(data,
                                                                        labels,
                                                                        random_state=random_state)
    
    model = make_pipeline(vectorizer, algorithm)
    model.fit(train_data, train_target)
    predicted = model.predict(test_data)
    return accuracy_score(predicted, test_target)

All Comments
----------

In [8]:
data = comm.commentBody
sections = comm.sectionName

In [11]:
random_state = 28

In [10]:
nbModel = MultinomialNB()

In [11]:
vectorizer = CountVectorizer(stop_words='english')
vec_train_test(data, sections, vectorizer, nbModel, random_state)

0.6418293548338392

In [12]:
vectorizer = CountVectorizer()
vec_train_test(data, sections, vectorizer, nbModel, random_state)

0.6359408736090351

In [13]:
vectorizer = CountVectorizer(input='content',
                             stop_words='english',
                             ngram_range=(1,2))
vec_train_test(data, sections, vectorizer, nbModel, random_state)

0.6076761637299753

In [14]:
vectorizer = CountVectorizer(input='content',
                             stop_words='english',
                             ngram_range=(1,3))
vec_train_test(data, sections, vectorizer, nbModel, random_state)

0.5936042034696744

In [15]:
vectorizer = TfidfVectorizer()
vec_train_test(data, sections, vectorizer, nbModel, random_state)

0.5884857544050369

In [16]:
vectorizer = TfidfVectorizer(stop_words='english')
vec_train_test(data, sections, vectorizer, nbModel, random_state)

0.5957633132521025

In [17]:
vectorizer = TfidfVectorizer(input='content', stop_words='english', ngram_range=(1,2))
vec_train_test(data, sections, vectorizer, nbModel, random_state)

0.5613081487520949

In [18]:
vectorizer = TfidfVectorizer(input='content', stop_words='english', ngram_range=(1,3))
vec_train_test(data, sections, vectorizer, nbModel, random_state)

0.5601002551675197

In [19]:
def tokenize(text):
    """
    Tokenize text and return a non-unique list of tokenized words
    found in the text. Normalize to lowercase, strip punctuation,
    remove stop words, drop words of length < 3, strip digits.
    """
    regex = re.compile('[' + re.escape(string.punctuation) + '0-9\\r\\t\\n]')
    nopunct = regex.sub(" ", text)  # delete stuff but leave at least a space to avoid clumping together
    words = nopunct.split(" ")
    words = [w for w in words if len(w) > 2]  # ignore a, an, to, at, be, ...
    words = [w.lower() for w in words]
    goodwords = [w for w in words if w not in ENGLISH_STOP_WORDS]
    return goodwords


def stemwords(words):
    """
    Given a list of tokens/words, return a new list with each word
    stemmed using a PorterStemmer.
    """
    stemmer = PorterStemmer()
    return [stemmer.stem(w) for w in words]


def tokenizer(text):
    return stemwords(tokenize(text))

In [20]:
vectorizer = TfidfVectorizer(input = 'content',
                            analyzer = 'word',
                            tokenizer = tokenizer,
                            stop_words = 'english',
                            decode_error = 'ignore')

In [21]:
vec_train_test(data, sections, vectorizer, nbModel, random_state)

0.5954613398559587

Remove Unknown Category
----------

In [47]:
filt = comm.sectionName != "Unknown"

In [48]:
data = comm.commentBody[filt]
sections = comm.sectionName[filt]

In [9]:
nbModel = MultinomialNB()

In [24]:
vectorizer = CountVectorizer(stop_words='english')
vec_train_test(data, sections, vectorizer, nbModel, random_state)

0.6808814527647419

In [25]:
vectorizer = CountVectorizer()
vec_train_test(data, sections, vectorizer, nbModel, random_state)

0.6664626266748283

In [26]:
vectorizer = CountVectorizer(input='content',
                             stop_words='english',
                             ngram_range=(1,2))
vec_train_test(data, sections, vectorizer, nbModel, random_state)

0.5884173297966402

In [27]:
vectorizer = CountVectorizer(input='content',
                             stop_words='english',
                             ngram_range=(1,3))
vec_train_test(data, sections, vectorizer, nbModel, random_state)

0.5703257838536353

In [28]:
vectorizer = TfidfVectorizer()
vec_train_test(data, sections, vectorizer, nbModel, random_state)

0.5306400054410665

In [29]:
vectorizer = TfidfVectorizer(stop_words='english')
vec_train_test(data, sections, vectorizer, nbModel, random_state)

0.5509759912942936

In [30]:
vectorizer = TfidfVectorizer(input='content', stop_words='english', ngram_range=(1,2))
vec_train_test(data, sections, vectorizer, nbModel, random_state)

0.4998979800040808

In [31]:
vectorizer = TfidfVectorizer(input='content', stop_words='english', ngram_range=(1,3))
vec_train_test(data, sections, vectorizer, nbModel, random_state)

0.498129633408148

In [32]:
vectorizer = TfidfVectorizer(input = 'content',
                            analyzer = 'word',
                            tokenizer = tokenizer,
                            stop_words = 'english',
                            decode_error = 'ignore')

In [33]:
vec_train_test(data, sections, vectorizer, nbModel, random_state)

0.5560429844249473

Filter to Comments Only without Unknowns
----------

In [6]:
filt = (comm.sectionName != "Unknown") & (comm.commentType == "comment")

In [25]:
data = comm.commentBody[filt]
sections = comm.sectionName[filt]

In [26]:
vectorizer = CountVectorizer(stop_words='english')
vec_train_test(data, sections, vectorizer, nbModel, random_state)

0.708839422933915